In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_metric

In [3]:
wer_metric = load_metric("wer")

In [4]:
test_df = pd.read_csv("../results/raw/intron-open-test-all_models.csv")
dev_df = pd.read_csv("../results/raw/intron-open-dev-all_models.csv")

test_norm_df = pd.read_csv("../results/normalized/intron-open-test-all_models.csv")
dev_norm_df = pd.read_csv("../results/normalized//intron-open-dev-all_models.csv")

In [5]:
df = pd.concat([dev_df, test_df])
norm_df = pd.concat([dev_norm_df, test_norm_df])

print(df.shape)
df.tail(2)

(167133, 14)


,idx,domain,gender,duration,age_group,accent,user_ids,audio_paths,origin,reference,prediction,wer,name,split
103947,475636,general,Female,7.237,26-40,tswana,ed4d0a0d0446ad1ff0da31efac2b7578,/AfriSpeech-100/test/1144d8ac-acd3-4a16-ba7d-6...,african,International Day for Mangrove: Warri to host ...,International day for mangrove colon worried t...,0.777778,AWS [Medical] (Primary Care),test
103948,712100,general,Female,16.147,19-25,ibibio,5856f88ae52888c338fe3ac15d673849,/AfriSpeech-100/test/76521f8a-6968-4ba7-ac3f-e...,nigerian,He said this on Wednesday night at the inaugur...,He said this on wednesday night at the inaugur...,0.555556,AWS [Medical] (Primary Care),test


- compute per-accent statistics
- compute per-gender statistics
- compute per-domain statistics
- compute per age-group statistics

## Table 4

In [6]:
# Sanity check

x = norm_df[(norm_df.split == "test") & (norm_df.name == "openai/whisper-large") & (norm_df.domain == "clinical")]
print(wer_metric.compute(predictions=x.prediction, references=x.reference))

x = norm_df[(df.split == "test") & (norm_df.name == "openai/whisper-large") & (norm_df.domain == "general")]
print(wer_metric.compute(predictions=x.prediction, references=x.reference))

x = norm_df[(norm_df.split == "test") & (norm_df.name == "openai/whisper-large")]
print(wer_metric.compute(predictions=x.prediction, references=x.reference))

0.37495623131413797
0.24034761334386706
0.30579053505486736


In [7]:
def domain_wer(df):
    df.domain = df.domain.str.capitalize()
    df.split = df.split.str.capitalize()
    
    # compute the domain WER
    domain_df = df.groupby(["name", "split", "domain"]).apply(lambda x: wer_metric.compute(predictions=x.prediction, references=x.reference))
    _domain_df = domain_df.reset_index().set_index("name").pivot(columns=["split", "domain"])
    _domain_df = _domain_df.droplevel(0, axis=1)
    
    # compute WER across the samples
    domain_df_ = df.groupby(["name", "split"]).apply(lambda x: wer_metric.compute(predictions=x.prediction, references=x.reference)).reset_index()
    domain_df_["domain"] = "Both"
    domain_df_ = domain_df_.set_index("name").pivot(columns=["split", "domain"])
    domain_df_ = domain_df_.droplevel(0, axis=1)
    
    # merge
    domain_df__ = _domain_df.merge(domain_df_, on="name").sort_values(("Test", "Both"))
    
    domain_df__ = domain_df__[[("Dev", "General"), ("Dev", "Clinical"), ("Dev", "Both"), ("Test", "General"), ("Test", "Clinical"), ("Test", "Both")]]
    domain_df__ = domain_df__.round(3)
    domain_df__.columns.names = ["", ""]
    
    return domain_df__


In [8]:
domain_df = domain_wer(df)
domain_df

Dev                    Test  \
                                            General Clinical   Both General   
name                                                                          
openai/whisper-large                          0.337    0.407  0.372   0.314   
openai/whisper-medium                         0.332    0.403  0.368   0.366   
openai/whisper-medium-en                      0.334    0.396  0.365   0.378   
Azure                                         0.508    0.538  0.524   0.426   
openai/whisper-small                          0.393    0.463  0.428   0.415   
openai/whisper-small-en                       0.384    0.460  0.422   0.417   
AWS                                           0.449    0.547  0.498   0.467   
AWS [Medical] (Primary Care)                  0.507    0.537  0.522   0.554   
GCP [Medical]                                 0.591    0.537  0.564   0.605   
GCP                                           0.538    0.615  0.577   0.594   
facebook/wav2vec2-large-xlsr-53-english       0.567    0.624  0.596   0.578   
facebook/wav2vec2-xls-r-1b-english            0.591    0.651  0.621   0.609   
microsoft/wavlm-libri-clean-100h-large        0.673    0.737  0.705   0.691   
microsoft/wavlm-libri-clean-100h-base         0.815    0.860  0.838   0.839   
facebook/hubert-large-ls960-ft                1.141    1.160  1.150   1.135   
facebook/wav2vec2-large-960h-lv60-self        1.149    1.165  1.157   1.141   
facebook/wav2vec2-large-960h                  1.148    1.183  1.166   1.146   
facebook/hubert-xlarge-ls960-ft               1.156    1.177  1.167   1.152   
facebook/wav2vec2-large-robust-ft-swbd-300h   1.191    1.224  1.208   1.194   
nvidia/nemo-conformer-ctc-large               0.527    0.588  0.557     NaN   
nvidia/nemo-conformer-transducer-large        0.538    0.589  0.564     NaN   
speechbrain/crdnn-rnnlm-librispeech           1.264    1.305  1.284     NaN   

                                                             
                                            Clinical   Both  
name                                                         
openai/whisper-large                           0.471  0.390  
openai/whisper-medium                          0.504  0.433  
openai/whisper-medium-en                       0.504  0.439  
Azure                                          0.528  0.476  
openai/whisper-small                           0.552  0.481  
openai/whisper-small-en                        0.565  0.489  
AWS                                            0.646  0.554  
AWS [Medical] (Primary Care)                   0.642  0.597  
GCP [Medical]                                  0.612  0.609  
GCP                                            0.680  0.636  
facebook/wav2vec2-large-xlsr-53-english        0.719  0.646  
facebook/wav2vec2-xls-r-1b-english             0.753  0.679  
microsoft/wavlm-libri-clean-100h-large         0.837  0.762  
microsoft/wavlm-libri-clean-100h-base          0.933  0.885  
facebook/hubert-large-ls960-ft                 1.188  1.161  
facebook/wav2vec2-large-960h-lv60-self         1.192  1.166  
facebook/wav2vec2-large-960h                   1.210  1.177  
facebook/hubert-xlarge-ls960-ft                1.212  1.181  
facebook/wav2vec2-large-robust-ft-swbd-300h    1.272  1.232  
nvidia/nemo-conformer-ctc-large                  NaN    NaN  
nvidia/nemo-conformer-transducer-large           NaN    NaN  
speechbrain/crdnn-rnnlm-librispeech              NaN    NaN

In [9]:
print(domain_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Dev} & \multicolumn{3}{l}{Test} \\
{} & General & Clinical &   Both & General & Clinical &   Both \\
name                                        &         &          &        &         &          &        \\
\midrule
openai/whisper-large                        &   0.337 &    0.407 &  0.372 &   0.314 &    0.471 &  0.390 \\
openai/whisper-medium                       &   0.332 &    0.403 &  0.368 &   0.366 &    0.504 &  0.433 \\
openai/whisper-medium-en                    &   0.334 &    0.396 &  0.365 &   0.378 &    0.504 &  0.439 \\
Azure                                       &   0.508 &    0.538 &  0.524 &   0.426 &    0.528 &  0.476 \\
openai/whisper-small                        &   0.393 &    0.463 &  0.428 &   0.415 &    0.552 &  0.481 \\
openai/whisper-small-en                     &   0.384 &    0.460 &  0.422 &   0.417 &    0.565 &  0.489 \\
AWS                                         &   0.449 &    0.547 &  0.498 &   0.467

In [10]:
domain_norm_df = domain_wer(norm_df)
domain_norm_df

Dev                    Test  \
                                            General Clinical   Both General   
name                                                                          
openai/whisper-large                          0.271    0.328  0.300   0.240   
openai/whisper-medium                         0.246    0.300  0.273   0.276   
openai/whisper-medium-en                      0.267    0.315  0.291   0.304   
openai/whisper-small                          0.313    0.372  0.343   0.330   
Azure                                         0.438    0.468  0.453   0.340   
openai/whisper-small-en                       0.319    0.384  0.352   0.350   
AWS                                           0.332    0.437  0.385   0.354   
AWS [Medical] (Primary Care)                  0.385    0.416  0.400   0.439   
GCP [Medical]                                 0.550    0.475  0.512   0.567   
facebook/wav2vec2-large-xlsr-53-english       0.498    0.561  0.530   0.506   
GCP                                           0.492    0.564  0.528   0.534   
facebook/wav2vec2-xls-r-1b-english            0.502    0.571  0.537   0.521   
facebook/wav2vec2-large-960h-lv60-self        0.512    0.587  0.550   0.533   
facebook/hubert-large-ls960-ft                0.536    0.598  0.567   0.557   
facebook/hubert-xlarge-ls960-ft               0.531    0.610  0.571   0.562   
microsoft/wavlm-libri-clean-100h-large        0.606    0.679  0.643   0.631   
facebook/wav2vec2-large-960h                  0.610    0.695  0.652   0.641   
facebook/wav2vec2-large-robust-ft-swbd-300h   0.689    0.778  0.734   0.733   
microsoft/wavlm-libri-clean-100h-base         0.782    0.834  0.808   0.811   
nvidia/nemo-conformer-ctc-large               0.410    0.486  0.448     NaN   
nvidia/nemo-conformer-transducer-large        0.408    0.477  0.443     NaN   
speechbrain/crdnn-rnnlm-librispeech           0.819    0.899  0.859     NaN   

                                                             
                                            Clinical   Both  
name                                                         
openai/whisper-large                           0.375  0.306  
openai/whisper-medium                          0.392  0.332  
openai/whisper-medium-en                       0.414  0.358  
openai/whisper-small                           0.455  0.391  
Azure                                          0.444  0.391  
openai/whisper-small-en                        0.482  0.414  
AWS                                            0.536  0.442  
AWS [Medical] (Primary Care)                   0.520  0.478  
GCP [Medical]                                  0.537  0.552  
facebook/wav2vec2-large-xlsr-53-english        0.650  0.576  
GCP                                            0.624  0.578  
facebook/wav2vec2-xls-r-1b-english             0.670  0.594  
facebook/wav2vec2-large-960h-lv60-self         0.694  0.611  
facebook/hubert-large-ls960-ft                 0.713  0.633  
facebook/hubert-xlarge-ls960-ft                0.725  0.641  
microsoft/wavlm-libri-clean-100h-large         0.783  0.705  
facebook/wav2vec2-large-960h                   0.797  0.717  
facebook/wav2vec2-large-robust-ft-swbd-300h    0.906  0.817  
microsoft/wavlm-libri-clean-100h-base          0.911  0.859  
nvidia/nemo-conformer-ctc-large                  NaN    NaN  
nvidia/nemo-conformer-transducer-large           NaN    NaN  
speechbrain/crdnn-rnnlm-librispeech              NaN    NaN

In [11]:
print(domain_norm_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Dev} & \multicolumn{3}{l}{Test} \\
{} & General & Clinical &   Both & General & Clinical &   Both \\
name                                        &         &          &        &         &          &        \\
\midrule
openai/whisper-large                        &   0.271 &    0.328 &  0.300 &   0.240 &    0.375 &  0.306 \\
openai/whisper-medium                       &   0.246 &    0.300 &  0.273 &   0.276 &    0.392 &  0.332 \\
openai/whisper-medium-en                    &   0.267 &    0.315 &  0.291 &   0.304 &    0.414 &  0.358 \\
openai/whisper-small                        &   0.313 &    0.372 &  0.343 &   0.330 &    0.455 &  0.391 \\
Azure                                       &   0.438 &    0.468 &  0.453 &   0.340 &    0.444 &  0.391 \\
openai/whisper-small-en                     &   0.319 &    0.384 &  0.352 &   0.350 &    0.482 &  0.414 \\
AWS                                         &   0.332 &    0.437 &  0.385 &   0.354

## Table 5

In [12]:
df.head(2)

,idx,domain,gender,duration,age_group,accent,user_ids,audio_paths,origin,reference,prediction,wer,name,split
0,72663,Clinical,Female,16.421995,19-25,swahili,f559cb4f16bc465ea44b56a8d3b5513e,/AfriSpeech-100/dev/e809b58c-4f05-4754-b98c-fb...,african,Ensure the correct placement of the feeding tu...,Ensure the correct placement of the feeding tu...,0.352941,openai/whisper-small,Dev
1,153902,Clinical,Female,11.393991,19-25,swahili,2be9bc423e70a24c703f8336e60af3f3,/AfriSpeech-100/dev/7ce31a48-f507-4344-bc49-96...,african,"Regardless of how amyocyteis stimulated, howev...","Regardless of how a moeocatase stimulated, how...",0.090909,openai/whisper-small,Dev


In [13]:
devset = pd.read_csv("../data/intron-dev-public-3231-clean.csv")
devset.head(2)

,idx,user_ids,accent,age_group,country,transcript,nchars,audio_ids,audio_paths,duration,origin,domain,split,gender
0,155349,659e36c14baaf7fa7bb197f951251f4b,setswana,26-40,BW,We should be asking ourselves whether we want ...,110,86ad3ef21e469217f28a749c990c81fd,/AfriSpeech-100/dev/92d2b94e-3e31-40be-b479-50...,8.400998,african,general,dev,Male
1,60812,1fd5f717cede9a867bf37d03c7d2166b,siswati,26-40,ZA,Other sagittal planes parallel to this off cen...,88,5409734bcc4a0f053e73f405d0f135da,/AfriSpeech-100/dev/127bdba1-8bc8-44a4-9c37-8e...,5.236984,african,clinical,dev,Female


In [14]:
devset.groupby(["accent"])["idx"].apply(lambda x: len(x)).sort_values(ascending=False)[:10]

accent
yoruba       361
swahili      313
zulu         310
setswana     216
igbo         216
twi          187
isizulu      169
luganda      149
hausa        116
afrikaans     82
Name: idx, dtype: int64

In [15]:
top10_nigeria = devset[devset["origin"] == "nigerian"].groupby(["accent"])["idx"].apply(lambda x: len(x)).sort_values(ascending=False)[:10]
top10_nigeria

accent
yoruba     361
igbo       216
hausa      116
idoma       50
ijaw        48
urhobo      44
english     44
ikwere      40
nupe        39
izon        34
Name: idx, dtype: int64

In [16]:
top10_nigeria.index.to_list()

['yoruba',
 'igbo',
 'hausa',
 'idoma',
 'ijaw',
 'urhobo',
 'english',
 'ikwere',
 'nupe',
 'izon']

In [17]:
top10_african = devset[devset["origin"] == "african"].groupby(["accent"])["idx"].apply(lambda x: len(x)).sort_values(ascending=False)[:10]
top10_african

accent
swahili      313
zulu         310
setswana     216
twi          187
isizulu      169
luganda      149
afrikaans     82
sepedi        81
siswati       62
tswana        56
Name: idx, dtype: int64

In [18]:
def accent_wer(df, top_accent_df, open_source_list, commercial_list, ours_list):
    df = df[df.accent.isin(top_accent_df.index.to_list())]
    print(df.shape)
    accent_df = df.groupby(["name", "accent"]).apply(lambda x: wer_metric.compute(predictions=x.prediction, references=x.reference))
    
    accent_df_ = accent_df.reset_index(name="WER").set_index("accent").pivot(columns="name")
    accent_df_ = accent_df_.droplevel(0, axis=1)
    
    samples = top_accent_df.to_frame().rename(columns = {"idx": "Samples"})
    
    accent_combined = pd.concat([samples, accent_df_[open_source], accent_df_[commercial], accent_df_[ours_list]], axis=1)
    
    return accent_combined.round(3)
    

In [19]:
open_source = ["facebook/wav2vec2-large-xlsr-53-english",
               "nvidia/nemo-conformer-ctc-large",
               "openai/whisper-medium"]
commercial = ["Azure", "GCP", "AWS"]

In [20]:
nigerian_dev_accent_df = accent_wer(dev_df, top10_nigeria, open_source, commercial, ours_list=[])
nigerian_dev_accent_df

(19558, 14)


,Samples,facebook/wav2vec2-large-xlsr-53-english,nvidia/nemo-conformer-ctc-large,openai/whisper-medium,Azure,GCP,AWS
accent,,,,,,,
yoruba,361,0.645,0.604,0.421,0.447,0.643,0.540
igbo,216,0.632,0.600,0.411,0.449,0.615,0.541
hausa,116,0.677,0.661,0.431,0.651,0.645,0.582
idoma,50,0.732,0.727,0.430,1.010,0.662,0.628
ijaw,48,0.672,0.661,0.420,0.865,0.670,0.550
urhobo,44,0.671,0.633,0.511,1.078,0.716,0.570
english,44,0.702,0.624,0.490,1.150,0.624,0.560
ikwere,40,0.523,0.434,0.323,1.020,0.535,0.399
nupe,39,0.724,0.590,0.440,1.126,0.763,0.549


In [21]:
print(nigerian_dev_accent_df.to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &  Samples &  facebook/wav2vec2-large-xlsr-53-english &  nvidia/nemo-conformer-ctc-large &  openai/whisper-medium &  Azure &    GCP &    AWS \\
accent  &          &                                          &                                  &                        &        &        &        \\
\midrule
yoruba  &      361 &                                    0.645 &                            0.604 &                  0.421 &  0.447 &  0.643 &  0.540 \\
igbo    &      216 &                                    0.632 &                            0.600 &                  0.411 &  0.449 &  0.615 &  0.541 \\
hausa   &      116 &                                    0.677 &                            0.661 &                  0.431 &  0.651 &  0.645 &  0.582 \\
idoma   &       50 &                                    0.732 &                            0.727 &                  0.430 &  1.010 &  0.662 &  0.628 \\
ijaw    &       48 &                             

# Finetuning

In [22]:
test_finetune_df = pd.read_csv("../results/raw/intron-open-test-all_models_finetuned.csv")
dev_finetune_df = pd.read_csv("../results/raw/intron-open-dev-all_models_finetuned.csv")

test_norm_finetune_df = pd.read_csv("../results/normalized/intron-open-test-all_models_finetuned.csv")
dev_norm_finetune_df = pd.read_csv("../results/normalized/intron-open-dev-all_models_finetuned.csv")

In [23]:
finetune_df = pd.concat([dev_finetune_df, test_finetune_df])
finetune_norm_df = pd.concat([dev_norm_finetune_df, test_norm_finetune_df])

In [24]:
domain_finetune_df = domain_wer(finetune_df)
domain_finetune_df

Dev                  \
                                                 General Clinical   Both   
name                                                                       
openai/whisper-medium-all                          0.327    0.344  0.335   
facebook/wav2vec2-large-xlsr-53-english-all          NaN      NaN    NaN   
openai/whisper-medium-clinical                     0.508    0.294  0.400   
facebook/wav2vec2-large-xlsr-53-english-general      NaN      NaN    NaN   
openai/whisper-medium-general                      0.321    0.551  0.437   
facebook/wav2vec2-large-xlsr-53-english-clinical     NaN      NaN    NaN   

                                                    Test                  
                                                 General Clinical   Both  
name                                                                      
openai/whisper-medium-all                          0.309    0.377  0.342  
facebook/wav2vec2-large-xlsr-53-english-all        0.374    0.428  0.400  
openai/whisper-medium-clinical                     0.482    0.315  0.401  
facebook/wav2vec2-large-xlsr-53-english-general    0.338    0.548  0.440  
openai/whisper-medium-general                      0.306    0.599  0.449  
facebook/wav2vec2-large-xlsr-53-english-clinical   0.487    0.429  0.459

In [25]:
print(domain_finetune_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Dev} & \multicolumn{3}{l}{Test} \\
{} & General & Clinical &   Both & General & Clinical &   Both \\
name                                             &         &          &        &         &          &        \\
\midrule
openai/whisper-medium-all                        &   0.327 &    0.344 &  0.335 &   0.309 &    0.377 &  0.342 \\
facebook/wav2vec2-large-xlsr-53-english-all      &     NaN &      NaN &    NaN &   0.374 &    0.428 &  0.400 \\
openai/whisper-medium-clinical                   &   0.508 &    0.294 &  0.400 &   0.482 &    0.315 &  0.401 \\
facebook/wav2vec2-large-xlsr-53-english-general  &     NaN &      NaN &    NaN &   0.338 &    0.548 &  0.440 \\
openai/whisper-medium-general                    &   0.321 &    0.551 &  0.437 &   0.306 &    0.599 &  0.449 \\
facebook/wav2vec2-large-xlsr-53-english-clinical &     NaN &      NaN &    NaN &   0.487 &    0.429 &  0.459 \\
\bottomrule
\end{tabular}



In [26]:
domain_finetune_norm_df = domain_wer(finetune_norm_df)
domain_finetune_norm_df

Dev                  \
                                                 General Clinical   Both   
name                                                                       
openai/whisper-medium-all                          0.213    0.241  0.227   
facebook/wav2vec2-large-xlsr-53-english-all          NaN      NaN    NaN   
facebook/wav2vec2-large-xlsr-53-english-general      NaN      NaN    NaN   
openai/whisper-medium-general                      0.205    0.487  0.347   
facebook/wav2vec2-large-xlsr-53-english-clinical     NaN      NaN    NaN   
openai/whisper-medium-clinical                     0.491    0.264  0.376   

                                                    Test                  
                                                 General Clinical   Both  
name                                                                      
openai/whisper-medium-all                          0.192    0.242  0.216  
facebook/wav2vec2-large-xlsr-53-english-all        0.279    0.308  0.293  
facebook/wav2vec2-large-xlsr-53-english-general    0.236    0.468  0.349  
openai/whisper-medium-general                      0.186    0.525  0.351  
facebook/wav2vec2-large-xlsr-53-english-clinical   0.424    0.308  0.368  
openai/whisper-medium-clinical                     0.464    0.266  0.368

In [27]:
print(domain_finetune_norm_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Dev} & \multicolumn{3}{l}{Test} \\
{} & General & Clinical &   Both & General & Clinical &   Both \\
name                                             &         &          &        &         &          &        \\
\midrule
openai/whisper-medium-all                        &   0.213 &    0.241 &  0.227 &   0.192 &    0.242 &  0.216 \\
facebook/wav2vec2-large-xlsr-53-english-all      &     NaN &      NaN &    NaN &   0.279 &    0.308 &  0.293 \\
facebook/wav2vec2-large-xlsr-53-english-general  &     NaN &      NaN &    NaN &   0.236 &    0.468 &  0.349 \\
openai/whisper-medium-general                    &   0.205 &    0.487 &  0.347 &   0.186 &    0.525 &  0.351 \\
facebook/wav2vec2-large-xlsr-53-english-clinical &     NaN &      NaN &    NaN &   0.424 &    0.308 &  0.368 \\
openai/whisper-medium-clinical                   &   0.491 &    0.264 &  0.376 &   0.464 &    0.266 &  0.368 \\
\bottomrule
\end{tabular}



In [28]:
# XLR, Whisper, Nemo
# General, CLinical, All

In [29]:
ours = ["openai/whisper-medium-all"]

In [30]:
nigerian_dev_accent_df = accent_wer(pd.concat([dev_df, dev_finetune_df]), top10_nigeria, open_source, commercial, ours)
nigerian_dev_accent_df

(22237, 14)


,Samples,facebook/wav2vec2-large-xlsr-53-english,nvidia/nemo-conformer-ctc-large,openai/whisper-medium,Azure,GCP,AWS,openai/whisper-medium-all
accent,,,,,,,,
yoruba,361,0.645,0.604,0.421,0.447,0.643,0.540,0.333
igbo,216,0.632,0.600,0.411,0.449,0.615,0.541,0.330
hausa,116,0.677,0.661,0.431,0.651,0.645,0.582,0.361
idoma,50,0.732,0.727,0.430,1.010,0.662,0.628,0.303
ijaw,48,0.672,0.661,0.420,0.865,0.670,0.550,0.352
urhobo,44,0.671,0.633,0.511,1.078,0.716,0.570,0.359
english,44,0.702,0.624,0.490,1.150,0.624,0.560,0.499
ikwere,40,0.523,0.434,0.323,1.020,0.535,0.399,0.242
nupe,39,0.724,0.590,0.440,1.126,0.763,0.549,0.293


In [31]:
nigerian_dev_norm_accent_df = accent_wer(pd.concat([dev_norm_df, dev_norm_finetune_df]), top10_nigeria, open_source, commercial, ours)
nigerian_dev_norm_accent_df



(22237, 14)


,Samples,facebook/wav2vec2-large-xlsr-53-english,nvidia/nemo-conformer-ctc-large,openai/whisper-medium,Azure,GCP,AWS,openai/whisper-medium-all
accent,,,,,,,,
yoruba,361,0.583,0.507,0.324,0.371,0.597,0.442,0.214
igbo,216,0.573,0.482,0.285,0.342,0.566,0.425,0.205
hausa,116,0.615,0.571,0.348,0.599,0.599,0.503,0.230
idoma,50,0.662,0.613,0.326,0.948,0.613,0.504,0.140
ijaw,48,0.618,0.559,0.345,0.802,0.634,0.439,0.229
urhobo,44,0.604,0.553,0.419,1.048,0.683,0.490,0.239
english,44,0.542,0.433,0.340,1.140,0.478,0.419,0.267
ikwere,40,0.474,0.354,0.248,1.015,0.506,0.297,0.140
nupe,39,0.671,0.509,0.387,1.100,0.718,0.498,0.203


In [32]:
print(nigerian_dev_norm_accent_df.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  Samples &  facebook/wav2vec2-large-xlsr-53-english &  nvidia/nemo-conformer-ctc-large &  openai/whisper-medium &  Azure &    GCP &    AWS &  openai/whisper-medium-all \\
accent  &          &                                          &                                  &                        &        &        &        &                            \\
\midrule
yoruba  &      361 &                                    0.583 &                            0.507 &                  0.324 &  0.371 &  0.597 &  0.442 &                      0.214 \\
igbo    &      216 &                                    0.573 &                            0.482 &                  0.285 &  0.342 &  0.566 &  0.425 &                      0.205 \\
hausa   &      116 &                                    0.615 &                            0.571 &                  0.348 &  0.599 &  0.599 &  0.503 &                      0.230 \\
idoma   &       50 &                                   

In [33]:
african_dev_norm_accent_df = accent_wer(pd.concat([dev_norm_df, dev_norm_finetune_df]), top10_african, open_source, commercial, ours)
african_dev_norm_accent_df


(36165, 14)


,Samples,facebook/wav2vec2-large-xlsr-53-english,nvidia/nemo-conformer-ctc-large,openai/whisper-medium,Azure,GCP,AWS,openai/whisper-medium-all
accent,,,,,,,,
swahili,313,0.539,0.480,0.253,0.317,0.487,0.387,0.249
zulu,310,0.447,0.356,0.217,0.294,0.469,0.311,0.229
setswana,216,0.421,0.355,0.205,0.295,0.432,0.308,0.229
twi,187,0.549,0.486,0.323,0.361,0.509,0.409,0.240
isizulu,169,0.509,0.401,0.224,0.305,0.436,0.323,0.243
luganda,149,0.475,0.359,0.197,0.282,0.486,0.302,0.251
afrikaans,82,0.461,0.339,0.160,0.232,0.405,0.276,0.216
sepedi,81,0.528,0.427,0.253,0.270,0.474,0.329,0.223
siswati,62,0.522,0.437,0.295,0.467,0.582,0.435,0.277


In [34]:
print(african_dev_norm_accent_df.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  Samples &  facebook/wav2vec2-large-xlsr-53-english &  nvidia/nemo-conformer-ctc-large &  openai/whisper-medium &  Azure &    GCP &    AWS &  openai/whisper-medium-all \\
accent    &          &                                          &                                  &                        &        &        &        &                            \\
\midrule
swahili   &      313 &                                    0.539 &                            0.480 &                  0.253 &  0.317 &  0.487 &  0.387 &                      0.249 \\
zulu      &      310 &                                    0.447 &                            0.356 &                  0.217 &  0.294 &  0.469 &  0.311 &                      0.229 \\
setswana  &      216 &                                    0.421 &                            0.355 &                  0.205 &  0.295 &  0.432 &  0.308 &                      0.229 \\
twi       &      187 &                         